In [2]:
%load_ext autoreload
%autoreload 2
from utils import *
from datasets import load_dataset
from transformers import (
    HfArgumentParser, 
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer
)
from link import *

/home/mloet/miniconda3/envs/lang/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [88]:
with open('boolq_perturbed.json', 'r') as f:
  json_data = json.load(f)
bundles = convert_bundles_to_yn(json_data)
with open('pubmed_test.json', 'r') as f:
  pubmed_test = json.load(f)
pubmed = load_dataset('qiaojin/PubMedQA', 'pqa_labeled')
pubmedds = convert_pubmed_to_yn(pubmed, list(map(int, list(pubmed_test.keys()))))


Label distribution in train split:
yes: 276 (62.02%)
no: 169 (37.98%)
Total examples in train: 445

Label distribution in test split:
yes: 276 (62.02%)
no: 169 (37.98%)
Total examples in test: 445


In [3]:
# dataset = load_dataset('bigbio/bioasq_task_b')
#ds = load_dataset("kroshan/BioASQ")
with open('datasets/BioASQ-training12b/training12b_new.json', 'r') as f:
  bio_train_data = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B1_golden.json', 'r') as f:
  bio_test_data1 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B2_golden.json', 'r') as f:
  bio_test_data2 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B3_golden.json', 'r') as f:
  bio_test_data3 = json.load(f)
with open('datasets/Task12BGoldenEnriched/12B4_golden.json', 'r') as f:
  bio_test_data4 = json.load(f)
golden = [bio_test_data1, bio_test_data2, bio_test_data3, bio_test_data4]
biods = convert_bioasq_to_yn(bio_train_data, [])


In [97]:

with open('boolq_perturbed.json', 'r') as f:
  json_data = json.load(f)
contrast = convert_contrast_to_yn(json_data)

In [104]:
boolq = load_dataset("boolq")

In [110]:
convboolq = convert_boolq_to_yn(boolq)

In [4]:
biods

for split in biods.keys():
  labels = biods[split]['label']
  label_counts = {}
  for label in labels:
      label_counts[label] = label_counts.get(label, 0) + 1
  print(f"\nLabel distribution in {split} split:")
  for label, count in label_counts.items():
      print(f"{label}: {count} ({count/len(labels)*100:.2f}%)")
  print(f"Total examples in {split}: {len(labels)}")


Label distribution in train split:
yes: 1003 (73.91%)
no: 354 (26.09%)
Total examples in train: 1357


In [114]:
total_lens = 0
ab256 = 0
ab128 = 0
ab512 = 0
for ex in contrast['train']:
  lenex = len(ex['context'].split(' '))
  total_lens += lenex
  if(lenex>512):
    ab512+=1
  if(lenex>256):
    ab256+=1
  if(lenex>128):
    ab128+=1
print(total_lens/15942)
print(ab128)
print(ab256)
print(ab512)

2.426232593150169
85
0
0


In [73]:
new = {split: {key: pubmedds[split][0][key] for key in ['question', 'context', 'label', 'id']} for split in pubmedds.keys()}
new

{'train': {'question': 'Can tailored interventions increase mammography use among HMO women?',
  'context': 'Telephone counseling and tailored print communications have emerged as promising methods for promoting mammography screening. However, there has been little research testing, within the same randomized field trial, of the efficacy of these two methods compared to a high-quality usual care system for enhancing screening. This study addressed the question: Compared to usual care, is tailored telephone counseling more effective than tailored print materials for promoting mammography screening? Three-year randomized field trial. One thousand ninety-nine women aged 50 and older recruited from a health maintenance organization in North Carolina. Women were randomized to 1 of 3 groups: (1) usual care, (2) tailored print communications, and (3) tailored telephone counseling. Adherence to mammography screening based on self-reports obtained during 1995, 1996, and 1997. Compared to usual 

In [35]:
tokenizer = AutoTokenizer.from_pretrained('./output/yn_models/boolq-art')

train_dataset = biotrain['train']
train_dataset = train_dataset.select(range(128))

train_dataset = train_dataset.map(
    lambda x: prepare_dataset_yn(x, tokenizer, 128),
    batched=True,
    remove_columns=train_dataset.column_names
)

Map: 100%|██████████| 128/128 [00:00<00:00, 347.82 examples/s]


In [90]:
ex = 'All but two of the universities in the Russell Group are part of the Sutton Trusts group of 30 highly selective universities, the Sutton Trust 30 (the absent members being Queen Mary University of London and Queens University Belfast). The Sutton 13 group of the 13 most highly selective universities only includes one non-Russell Group member, the University of St Andrews. St Andrews was also the only non-Russell Group University in the top 10 by average UCAS tariff score of new undergraduate students in 2015--16, placing fifth with an average score of 525 (and an offer rate of 52.2%). Half of the Russell Group made offers to more than three quarter of their undergraduate applicants in 2015.'

In [91]:
len(ex.split(' '))

117

In [5]:
control = {
  "total_f1": 0.6236936577582768,
  "total_exact_match": 0.6426966292134831,
  "total_precision": 0.6229406049340993,
  "total_recall": 0.6247856101535032,
  "yes_f1": 0.708256880733945,
  "yes_precision": 0.7174721189591078,
  "yes_recall": 0.6992753623188406,
  "yes_exact_match": 0.4337078651685393,
  "no_f1": 0.5391304347826087,
  "no_precision": 0.5284090909090909,
  "no_recall": 0.5502958579881657,
  "no_exact_match": 0.20898876404494382
}

exp = {
  "total_f1": 0.6685127607203005,
  "total_exact_match": 0.6898876404494382,
  "total_precision": 0.6698864378018535,
  "total_recall": 0.6674170311294058,
  "yes_f1": 0.7526881720430108,
  "yes_precision": 0.7446808510638298,
  "yes_recall": 0.7608695652173914,
  "yes_exact_match": 0.47191011235955055,
  "no_f1": 0.5843373493975904,
  "no_precision": 0.5950920245398773,
  "no_recall": 0.5739644970414202,
  "no_exact_match": 0.21797752808988763
}

In [6]:
res = {key: exp[key] - control[key] for key in control.keys()}

In [7]:
res

{'total_f1': 0.04481910296202374,
 'total_exact_match': 0.04719101123595504,
 'total_precision': 0.04694583286775422,
 'total_recall': 0.042631420975902556,
 'yes_f1': 0.04443129130906576,
 'yes_precision': 0.027208732104721967,
 'yes_recall': 0.061594202898550776,
 'yes_exact_match': 0.03820224719101123,
 'no_f1': 0.04520691461498172,
 'no_precision': 0.06668293363078637,
 'no_recall': 0.023668639053254448,
 'no_exact_match': 0.008988764044943809}